# Memory Reduction

### Copied & Edited From FCC - Load Datasets - 75% Memory Reduction by Abdelghani Belgaid

This code will be copied into whatever other files I need it in, I think.

In [1]:
import re, gc
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None) # Just to be able to scroll all 1.2k columns
gc.enable() # For cleanup

## Training Set

In [33]:
# Reduce memory usage of a pandas DataFrame
def reduce_memory_usage(df):
    # Iterate through columns and modify data types to be minimum size
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage of DataFrame: {start_mem} MB')
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage after optimization: {end_mem} MB')
    print(f'Decreased by {100 * (start_mem - end_mem) / start_mem}%')
    return df


In [2]:
# Define file paths and names for maize train datasets
maize_train_datasets = [
    {'file_name': '', 'path': '/Users/james/FutureCrop_Kaggle/Data/soil_co2_maize_train.parquet'},
    {'file_name': 'pr_', 'path': '/Users/james/FutureCrop_Kaggle/Data/pr_maize_train.parquet'},
    {'file_name': 'rsds_', 'path': '/Users/james/FutureCrop_Kaggle/Data/rsds_maize_train.parquet'},
    {'file_name': 'tasmin_', 'path': '/Users/james/FutureCrop_Kaggle/Data/tasmin_maize_train.parquet'},
    {'file_name': 'tasmax_', 'path': '/Users/james/FutureCrop_Kaggle/Data/tasmax_maize_train.parquet'},
    {'file_name': 'tas_', 'path': '/Users/james/FutureCrop_Kaggle/Data/tas_maize_train.parquet'},
    {'file_name': '', 'path': '/Users/james/FutureCrop_Kaggle/Data/train_solutions_maize.parquet'},
]

# Read and store each train dataset with file name prefix
maize_train_dfs = [pd.read_parquet(file['path']).add_prefix(file['file_name']) for file in maize_train_datasets]

# Concatenate all maize train datasets horizontally
maize_train_df = pd.concat(maize_train_dfs, axis=1)
# maize_train_df = reduce_memory_usage(maize_train_df)

del maize_train_dfs
gc.collect()

# Repeat process for wheat train datasets
wheat_train_datasets = [
    {'file_name': '', 'path': '/Users/james/FutureCrop_Kaggle/Data/soil_co2_wheat_train.parquet'},
    {'file_name': 'pr_', 'path': '/Users/james/FutureCrop_Kaggle/Data/pr_wheat_train.parquet'},
    {'file_name': 'rsds_', 'path': '/Users/james/FutureCrop_Kaggle/Data/rsds_wheat_train.parquet'},
    {'file_name': 'tasmin_', 'path': '/Users/james/FutureCrop_Kaggle/Data/tasmin_wheat_train.parquet'},
    {'file_name': 'tasmax_', 'path': '/Users/james/FutureCrop_Kaggle/Data/tasmax_wheat_train.parquet'},
    {'file_name': 'tas_', 'path': '/Users/james/FutureCrop_Kaggle/Data/tas_wheat_train.parquet'},
    {'file_name': '', 'path': '/Users/james/FutureCrop_Kaggle/Data/train_solutions_wheat.parquet'},
]

wheat_train_dfs = [pd.read_parquet(file['path']).add_prefix(file['file_name']) for file in wheat_train_datasets]

wheat_train_df = pd.concat(wheat_train_dfs, axis=1)
# wheat_train_df = reduce_memory_usage(wheat_train_df)

del wheat_train_dfs
gc.collect()


0

In [3]:
# Combine the maize & wheat train dataframes 
train = pd.concat([maize_train_df, wheat_train_df], ignore_index=False)

del maize_train_df, wheat_train_df
gc.collect()

columns_to_drop = [
    'pr_crop', 'pr_year', 'pr_lon', 'pr_lat', 'pr_variable',
    'rsds_crop', 'rsds_year', 'rsds_lon', 'rsds_lat', 'rsds_variable',
    'tasmin_crop', 'tasmin_year', 'tasmin_lon', 'tasmin_lat', 'tasmin_variable',
    'tasmax_crop', 'tasmax_year', 'tasmax_lon', 'tasmax_lat', 'tasmax_variable',
    'tas_crop', 'tas_year', 'tas_lon', 'tas_lat', 'tas_variable',
]

train.drop(columns=columns_to_drop, axis=1, inplace=True)
    

In [4]:
train.head()

,crop,year,lon,lat,texture_class,real_year,co2,nitrogen,pr_0,pr_1,pr_2,pr_3,pr_4,pr_5,pr_6,pr_7,pr_8,pr_9,pr_10,pr_11,pr_12,pr_13,pr_14,pr_15,pr_16,pr_17,pr_18,pr_19,pr_20,pr_21,pr_22,pr_23,pr_24,pr_25,pr_26,pr_27,pr_28,pr_29,pr_30,pr_31,pr_32,pr_33,pr_34,pr_35,pr_36,pr_37,pr_38,pr_39,pr_40,pr_41,pr_42,pr_43,pr_44,pr_45,pr_46,pr_47,pr_48,pr_49,pr_50,pr_51,pr_52,pr_53,pr_54,pr_55,pr_56,pr_57,pr_58,pr_59,pr_60,pr_61,pr_62,pr_63,pr_64,pr_65,pr_66,pr_67,pr_68,pr_69,pr_70,pr_71,pr_72,pr_73,pr_74,pr_75,pr_76,pr_77,pr_78,pr_79,pr_80,pr_81,pr_82,pr_83,pr_84,pr_85,pr_86,pr_87,pr_88,pr_89,pr_90,pr_91,pr_92,pr_93,pr_94,pr_95,pr_96,pr_97,pr_98,pr_99,pr_100,pr_101,pr_102,pr_103,pr_104,pr_105,pr_106,pr_107,pr_108,pr_109,pr_110,pr_111,pr_112,pr_113,pr_114,pr_115,pr_116,pr_117,pr_118,pr_119,pr_120,pr_121,pr_122,pr_123,pr_124,pr_125,pr_126,pr_127,pr_128,pr_129,pr_130,pr_131,pr_132,pr_133,pr_134,pr_135,pr_136,pr_137,pr_138,pr_139,pr_140,pr_141,pr_142,pr_143,pr_144,pr_145,pr_146,pr_147,pr_148,pr_149,pr_150,pr_151,pr_152,pr_153,pr_154,pr_155,pr_156,pr_157,pr_158,pr_159,pr_160,pr_161,pr_162,pr_163,pr_164,pr_165,pr_166,pr_167,pr_168,pr_169,pr_170,pr_171,pr_172,pr_173,pr_174,pr_175,pr_176,pr_177,pr_178,pr_179,pr_180,pr_181,pr_182,pr_183,pr_184,pr_185,pr_186,pr_187,pr_188,pr_189,pr_190,pr_191,pr_192,pr_193,pr_194,pr_195,pr_196,pr_197,pr_198,pr_199,pr_200,pr_201,pr_202,pr_203,pr_204,pr_205,pr_206,pr_207,pr_208,pr_209,pr_210,pr_211,pr_212,pr_213,pr_214,pr_215,pr_216,pr_217,pr_218,pr_219,pr_220,pr_221,pr_222,pr_223,pr_224,pr_225,pr_226,pr_227,pr_228,pr_229,pr_230,pr_231,pr_232,pr_233,pr_234,pr_235,pr_236,pr_237,pr_238,pr_239,rsds_0,rsds_1,rsds_2,rsds_3,rsds_4,rsds_5,rsds_6,rsds_7,rsds_8,rsds_9,rsds_10,rsds_11,rsds_12,rsds_13,rsds_14,rsds_15,rsds_16,rsds_17,rsds_18,rsds_19,rsds_20,rsds_21,rsds_22,rsds_23,rsds_24,rsds_25,rsds_26,rsds_27,rsds_28,rsds_29,rsds_30,rsds_31,rsds_32,rsds_33,rsds_34,rsds_35,rsds_36,rsds_37,rsds_38,rsds_39,rsds_40,rsds_41,rsds_42,rsds_43,rsds_44,rsds_45,rsds_46,rsds_47,rsds_48,rsds_49,rsds_50,rsds_51,rsds_52,rsds_53,rsds_54,rsds_55,rsds_56,rsds_57,rsds_58,rsds_59,rsds_60,rsds_61,rsds_62,rsds_63,rsds_64,rsds_65,rsds_66,rsds_67,rsds_68,rsds_69,rsds_70,rsds_71,rsds_72,rsds_73,rsds_74,rsds_75,rsds_76,rsds_77,rsds_78,rsds_79,rsds_80,rsds_81,rsds_82,rsds_83,rsds_84,rsds_85,rsds_86,rsds_87,rsds_88,rsds_89,rsds_90,rsds_91,rsds_92,rsds_93,rsds_94,rsds_95,rsds_96,rsds_97,rsds_98,rsds_99,rsds_100,rsds_101,rsds_102,rsds_103,rsds_104,rsds_105,rsds_106,rsds_107,rsds_108,rsds_109,rsds_110,rsds_111,rsds_112,rsds_113,rsds_114,rsds_115,rsds_116,rsds_117,rsds_118,rsds_119,rsds_120,rsds_121,rsds_122,rsds_123,rsds_124,rsds_125,rsds_126,rsds_127,rsds_128,rsds_129,rsds_130,rsds_131,rsds_132,rsds_133,rsds_134,rsds_135,rsds_136,rsds_137,rsds_138,rsds_139,rsds_140,rsds_141,rsds_142,rsds_143,rsds_144,rsds_145,rsds_146,rsds_147,rsds_148,rsds_149,rsds_150,rsds_151,rsds_152,rsds_153,rsds_154,rsds_155,rsds_156,rsds_157,rsds_158,rsds_159,rsds_160,rsds_161,rsds_162,rsds_163,rsds_164,rsds_165,rsds_166,rsds_167,rsds_168,rsds_169,rsds_170,rsds_171,rsds_172,rsds_173,rsds_174,rsds_175,rsds_176,rsds_177,rsds_178,rsds_179,rsds_180,rsds_181,rsds_182,rsds_183,rsds_184,rsds_185,rsds_186,rsds_187,rsds_188,rsds_189,rsds_190,rsds_191,rsds_192,rsds_193,rsds_194,rsds_195,rsds_196,rsds_197,rsds_198,rsds_199,rsds_200,rsds_201,rsds_202,rsds_203,rsds_204,rsds_205,rsds_206,rsds_207,rsds_208,rsds_209,rsds_210,rsds_211,rsds_212,rsds_213,rsds_214,rsds_215,rsds_216,rsds_217,rsds_218,rsds_219,rsds_220,rsds_221,rsds_222,rsds_223,rsds_224,rsds_225,rsds_226,rsds_227,rsds_228,rsds_229,rsds_230,rsds_231,rsds_232,rsds_233,rsds_234,rsds_235,rsds_236,rsds_237,rsds_238,rsds_239,tasmin_0,tasmin_1,tasmin_2,tasmin_3,tasmin_4,tasmin_5,tasmin_6,tasmin_7,tasmin_8,tasmin_9,tasmin_10,tasmin_11,tasmin_12,tasmin_13,tasmin_14,tasmin_15,tasmin_16,tasmin_17,tasmin_18,tasmin_19,tasmin_20,tasmin_21,tasmin_22,tasmin_23,tasmin_24,tasmin_25,tasmin_26,tasmin_27,tasmin_28,tasmin_29,tasmin_30,tasmin_31,tasmin_32,tasm

In [29]:
train.shape[0]

628466

In [5]:
train.to_parquet('train.parquet')

## Test Set

In [6]:
# Define file paths and names for maize train datasets
maize_test_datasets = [
    {'file_name': '', 'path': '/Users/james/FutureCrop_Kaggle/Data/soil_co2_maize_test.parquet'},
    {'file_name': 'pr_', 'path': '/Users/james/FutureCrop_Kaggle/Data/pr_maize_test.parquet'},
    {'file_name': 'rsds_', 'path': '/Users/james/FutureCrop_Kaggle/Data/rsds_maize_test.parquet'},
    {'file_name': 'tasmin_', 'path': '/Users/james/FutureCrop_Kaggle/Data/tasmin_maize_test.parquet'},
    {'file_name': 'tasmax_', 'path': '/Users/james/FutureCrop_Kaggle/Data/tasmax_maize_test.parquet'},
    {'file_name': 'tas_', 'path': '/Users/james/FutureCrop_Kaggle/Data/tas_maize_test.parquet'}
]

# Read and store each train dataset with file name prefix
maize_test_dfs = [pd.read_parquet(file['path']).add_prefix(file['file_name']) for file in maize_test_datasets]

# Concatenate all maize train datasets horizontally
maize_test_df = pd.concat(maize_test_dfs, axis=1)
# maize_train_df = reduce_memory_usage(maize_train_df)

del maize_test_dfs
gc.collect()

# Repeat process for wheat train datasets
wheat_test_datasets = [
    {'file_name': '', 'path': '/Users/james/FutureCrop_Kaggle/Data/soil_co2_wheat_test.parquet'},
    {'file_name': 'pr_', 'path': '/Users/james/FutureCrop_Kaggle/Data/pr_wheat_test.parquet'},
    {'file_name': 'rsds_', 'path': '/Users/james/FutureCrop_Kaggle/Data/rsds_wheat_test.parquet'},
    {'file_name': 'tasmin_', 'path': '/Users/james/FutureCrop_Kaggle/Data/tasmin_wheat_test.parquet'},
    {'file_name': 'tasmax_', 'path': '/Users/james/FutureCrop_Kaggle/Data/tasmax_wheat_test.parquet'},
    {'file_name': 'tas_', 'path': '/Users/james/FutureCrop_Kaggle/Data/tas_wheat_test.parquet'}
]

wheat_test_dfs = [pd.read_parquet(file['path']).add_prefix(file['file_name']) for file in wheat_test_datasets]

wheat_test_df = pd.concat(wheat_test_dfs, axis=1)
# wheat_train_df = reduce_memory_usage(wheat_train_df)

del wheat_test_dfs
gc.collect()


0

In [7]:
# Combine the maize & wheat test dataframes 
test = pd.concat([maize_test_df, wheat_test_df], ignore_index=False)

del maize_test_df, wheat_test_df
gc.collect()

columns_to_drop = [
    'pr_crop', 'pr_year', 'pr_lon', 'pr_lat', 'pr_variable',
    'rsds_crop', 'rsds_year', 'rsds_lon', 'rsds_lat', 'rsds_variable',
    'tasmin_crop', 'tasmin_year', 'tasmin_lon', 'tasmin_lat', 'tasmin_variable',
    'tasmax_crop', 'tasmax_year', 'tasmax_lon', 'tasmax_lat', 'tasmax_variable',
    'tas_crop', 'tas_year', 'tas_lon', 'tas_lat', 'tas_variable',
]

test.drop(columns=columns_to_drop, axis=1, inplace=True)
  

In [8]:
test.to_parquet('test.parquet')